### Solution Note:

- Within `sandFallInfinite` I expand my boundary (just the column) outward by 1.
- This has the effect of taking a (10,10) matrix to be (10,12). 
- This also means I need to rebuild the base row to be all rock (since this is an "infinite" floor)
- And finally, I need to output the matrix each time a piece of sand falls
    - I then have to take the new shape and recalculate the starting index
- If the most recent output equals start then we solved it!

In [1]:
import numpy as np

with open("data/day14_sample.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")

def build_coords(l, r):
    """ Build V or H line based on input coords"""
    x1, y1 = [int(x) for x in l.split(',')]
    x2, y2 = [int(x) for x in r.split(',')]
    
    if x1 == x2:
        return [(y, x1) for y in range(min(y1, y2), max(y1, y2) + 1)]
    elif y1 == y2:
        return [(y1, x) for x in range(min(x1, x2), max(x1, x2) + 1)]
    else:
        raise ValueError("This shouldn't happen, you misunderstood the prompt!")
        
def flattenArray(l):
    return np.asarray([item for sublist in l for item in sublist])

def sandFallInfinite(mat, start):
    r, c = start
    b = np.pad(mat, pad_width=((0,0),(1,1)), mode='constant', constant_values='.')

    # fall until we hit something: an index error is no longer possible as infinite fall is gone
    while falling:
        # expand outwards as needed
        if (c - 1 == 0) or (c  == mat.shape[1] - 1):
            mat = np.pad(mat, pad_width=((0,0),(1,1)), mode='constant', constant_values='.')
            c += 1 # we need to update c when we expand since we go both directions

            # need to expand inifinite line
            mat[mat.shape[0] - 1, :] = '#'

        if mat[r + 1,c] not in ['#', 'O']: # fall downward until we hit a rock
            r += 1
        else:
            if mat[r + 1,c - 1] not in ['#', 'O']: # try diag left first
                r += 1
                c -= 1
            elif mat[r + 1, c + 1] not in ['#', 'O']: # try diag right
                r += 1
                c += 1
            else: # nowere else to go
                return (r,c, mat)

In [2]:
# get coords of rocks
rock_coords = []
for l in lines:
    vals = l.split(' -> ')
    for i in range(len(vals) - 1):
        rock_coords.append(build_coords(vals[i], vals[i+1]))

# single array of coords (written as row, column)
all_rocks = flattenArray(rock_coords)

# calculate boundaries and offset, which is used mainly
# for indexing / visualization
min_r = np.min(all_rocks[:,0])
max_r = np.max(all_rocks[:,0])
min_c = np.min(all_rocks[:,1])
max_c = np.max(all_rocks[:,1])

print(min_r, max_r)
print(min_c, max_c)
offset = min_c

# Build our matrix and look at starting rocks
mat = np.zeros((max_r + 1 + 2, max_c - min_c + 1), str)
mat[max_r + 2, :] = '#'
mat[mat == ''] = '.'
for rock in all_rocks:
    r, c = (rock[0], rock[1])
    mat[r, c - offset] = '#'
    
# And our starting point
sr , sc = (0, 500)
mat[sr, sc - offset] = '+'
    
mat

4 9
494 503


array([['.', '.', '.', '.', '.', '.', '+', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '#', '.', '.', '.', '#', '#'],
       ['.', '.', '.', '.', '#', '.', '.', '.', '#', '.'],
       ['.', '.', '#', '#', '#', '.', '.', '.', '#', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [3]:
# Run falling simulation
shift = 0
falling = True
start = (0, 500 - offset + shift)
i = 0
while falling:
    # matrix x-axis size prior to shift
    pre_size = mat.shape[1]
    r,c, mat = sandFallInfinite(mat, start)
    
    # need to do a check on the start, it will shift
    post_size = mat.shape[1]
    shift += (post_size - pre_size) / 2
    start = (0, 500 - offset + int(shift))
    
    if r == c == 0:
        print("skipped")

    mat[(r,c)] = 'O'
    if (r,c) == start:
        print("Hooray we blocked it")
        break
    
print(f"Total sand at rest: {len(np.argwhere(mat == 'O'))}")
for row in mat:
    print(''.join(list(row)))

Hooray we blocked it
Total sand at rest: 93
..............O...........
.............OOO..........
............OOOOO.........
...........OOOOOOO........
..........OO#OOO##O.......
.........OOO#OOO#OOO......
........OO###OOO#OOOO.....
.......OOOO.OOOO#OOOOO....
......OOOOOOOOOO#OOOOOO...
.....OOO#########OOOOOOO..
....OOOOO.......OOOOOOOOO.
##########################


In [4]:
# actual
with open("data/day14.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")

# get coords of rocks
rock_coords = []
for l in lines:
    vals = l.split(' -> ')
    for i in range(len(vals) - 1):
        rock_coords.append(build_coords(vals[i], vals[i+1]))

# single array of coords (written as row, column)
all_rocks = flattenArray(rock_coords)

# calculate boundaries and offset, which is used mainly
# for indexing / visualization
min_r = np.min(all_rocks[:,0])
max_r = np.max(all_rocks[:,0])
min_c = np.min(all_rocks[:,1])
max_c = np.max(all_rocks[:,1])

print(min_r, max_r)
print(min_c, max_c)
offset = min_c

# Build our matrix and look at starting rocks
mat = np.zeros((max_r + 1 + 2, max_c - min_c + 1), str)
mat[max_r + 2, :] = '#'
mat[mat == ''] = '.'
for rock in all_rocks:
    r, c = (rock[0], rock[1])
    mat[r, c - offset] = '#'
    
# And our starting point
sr , sc = (0, 500)
mat[sr, sc - offset] = '+'

# Run falling simulation
shift = 0
falling = True
start = (0, 500 - offset + shift)
i = 0
while falling:
    pre_size = mat.shape[1]
    r,c, mat = sandFallInfinite(mat, start)
    
    post_size = mat.shape[1]
    shift += (post_size - pre_size) / 2
    start = (0, 500 - offset + int(shift))
    
    if r == c == 0:
        print("skipped")

    mat[(r,c)] = 'O'
    if (r,c) == start:
        print("Hooray we blocked it")
        break
    
print(f"Total sand at rest: {len(np.argwhere(mat == 'O'))}")

13 165
368 518
Hooray we blocked it
Total sand at rest: 25585
